# [LADOT]((https://www.ladotbus.com/)) GTFS Realtime Vehicle positions

GTFS Realtime is a feed specification that allows public transportation agencies to provide realtime updates about their fleet to application developers. It is used globally by many transit agencies and is one of the widely adopted open data standards for public transportation industry.

We will be exploring Vehicle Positions from [LADOT (Los Angeles Department of Transportation)](https://www.ladotbus.com/).

## Imports

In [98]:
from google.transit import gtfs_realtime_pb2
from protobuf_to_dict import protobuf_to_dict
import requests
from collections import OrderedDict
import pandas as pd

## Urls

Vehicle Positions — information about the vehicles including location and congestion level in real time.

In [32]:
# gtfs_schedule_url = "https://ladotbus.com/gtfs"
ladot_vehicle_positions_url = "https://ladotbus.com/gtfs-rt/vehiclepositions"
#ladot_service_alerts_url = "https://ladotbus.com/gtfs-rt/alerts"
#ladot_trip_updates_url = "https://ladotbus.com/gtfs-rt/tripupdates"

## Request

We will download a GTFS-realtime data feed from our URL, parsing it as a [FeedMessage (the root type of the GTFS-realtime schema)](https://developers.google.com/transit/gtfs-realtime/reference), and iterating over the results. Code snippet adapted from [Google Transit APIs](https://developers.google.com/transit/gtfs-realtime/examples/python-sample)

In [83]:
# Get FeedMessage from url
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get(ladot_vehicle_positions_url)
feed.ParseFromString(response.content)

# Let's look at one entity
feed.entity[0]    

id: "vehicle_6771"
vehicle {
  trip {
    trip_id: "30-N-wZH7vIqkW"
    start_time: "20:15:00"
    start_date: "20220605"
    direction_id: 1
  }
  position {
    latitude: 34.122901916503906
    longitude: -118.2969970703125
    bearing: 353.76025390625
    speed: 8.046719551086426
  }
  timestamp: 1654488862
  vehicle {
    id: "6771"
    label: "15344"
  }
  occupancy_status: EMPTY
}

## Data

Each entity contains the following information:

- `id`
- `vehicle`
    - `trip`
        - `trip_id`
        - `start_time`
        - `start_date`
        - `direction_id`: DATETYPE, DESCRIPTION, EXAMPLES
    - `position`
        - `latitude`
        - `longitude`
        - `bearing`
        - `speed`
    - `timestamp`
    - `vehicle`
        - `id`
        - `label`
    - `occupancy_status`

## Convert feed to dataframe

We want to parse GTFS Real time Protobuf into more usable tabular format. Let's use a FOR LOOP to iterate over the nested dictionary structure and collect each rows and append & collect it in form of pandas dataframe.

In [90]:
dict_obj = protobuf_to_dict(feed)

In [124]:
collector = []

for block in dict_obj['entity']:
    row = OrderedDict()
    # id
    row['id'] = block['id']
    # vehicle blocks
    trip = block['vehicle']['trip']
    position = block['vehicle']['position']
    vehicle = block['vehicle']['vehicle']
    
    # trip
    row['trip_id'] = trip.get('trip_id','')
    row['start_time'] = trip.get('start_time','')
    row['start_date'] = trip.get('start_date','')
    row['direction_id'] = trip.get('direction_id','')
    # position 
    row['position'] = position.get('latitude','')
    row['longitude'] = position.get('longitude','')
    row['bearing'] = position.get('bearing','')
    row['speed'] = position.get('speed','')
    # timestamp
    row['timestamp'] = block['vehicle']['timestamp']
    # vehicle
    row['id'] = vehicle.get('id','')
    row['label'] = vehicle.get('label','')
    # occupancy_status
    row['occupancy_status'] = block['vehicle']['occupancy_status']
    
    collector.append(row)
    
df = pd.DataFrame(collector)

In [125]:
df

,id,trip_id,start_time,start_date,direction_id,position,longitude,bearing,speed,timestamp,label,occupancy_status
0,6771,30-N-wZH7vIqkW,20:15:00,20220605,1,34.122902,-118.296997,353.760254,8.04672,1654488862,15344,0
1,6454,30-7PPn9alwcS8r,20:35:00,20220605,0,34.087467,-118.174965,61.482147,1.78816,1654488894,09314,0
2,6449,30-6m6encTzZP5,18:37:00,20220605,1,34.081108,-118.254684,197.254410,7.15264,1654488896,09322,0
3,3805,30-fZ4AtJ0uD_x,18:50:00,20220605,0,34.018772,-118.237259,217.000000,0.00000,1654486958,17305,0
4,709,30-W4sYJNT-ThY,20:52:00,20220605,1,34.062260,-118.271133,117.911385,6.25856,1654488894,13325,0
5,4386,30-qInu5sBLJxb,20:52:00,20220605,1,34.069496,-118.181168,266.687836,7.59968,1654488897,15368,1
6,324,30-7RUxyh61_tI,21:15:00,20220605,1,34.104282,-118.291710,357.068024,9.38784,1654488894,15301,0
7,1631,30-dOInNHE3jv1,18:32:00,20220605,0,34.065090,-118.265495,27.337957,5.36448,1654488896,15343,1
8,708,30-aehmDrhJlQE,21:02:00,20220605,0,34.031616,-118.267914,44.000000,3.12928,1654488897,12335,0
9,1641,30-vKSNXOQkVnW,21:05:00,20220605,0,34.111412,-118.289818,77.405930,8.49376,1654488896,15346,0
